In [1]:
import numpy as np
import soundfile as sf
import librosa

import pyaudio
import wave

from tensorflow import keras

In [2]:
CHUNK = 1024 
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 1
RATE = 8000 #sample rate
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "testab.wav"

In [3]:
model = keras.models.load_model("voice_model_1.h5")
model.summary()

Model: "SYS_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 39, 49, 1)]       0         
_________________________________________________________________
CONV1 (Conv2D)               (None, 39, 49, 10)        60        
_________________________________________________________________
BATCH1 (BatchNormalization)  (None, 39, 49, 10)        40        
_________________________________________________________________
CONV2 (Conv2D)               (None, 39, 49, 1)         51        
_________________________________________________________________
BATCH2 (BatchNormalization)  (None, 39, 49, 1)         4         
_________________________________________________________________
dropout (Dropout)            (None, 39, 49, 1)         0         
_________________________________________________________________
SQZZ (Reshape)               (None, 39, 49)            0 

In [50]:
p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,channels=CHANNELS,rate=RATE,input=True,frames_per_buffer=CHUNK) #buffer
print("* recording")
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel
print("* done recording")
stream.stop_stream()
stream.close()
p.terminate()
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


data, samplerate = sf.read(WAVE_OUTPUT_FILENAME)
le = len(data)

new_data = np.zeros(25000)
new_data[100:100+le] = data

temp_mel = librosa.feature.melspectrogram(new_data, sr = 8000, n_fft = 1024, n_mels = 39, power = 2).reshape(1,39,49,1)


prediction = model.predict(temp_mel)

arg_val = np.argmax(prediction)

if (arg_val == 0):
    print("Forward")
elif (arg_val == 1):
    print("Back")
elif (arg_val == 2):
    print("Left")
elif (arg_val == 3):
    print("Right")
elif (arg_val == 4):
    print("Stop")

* recording
* done recording
Right


In [5]:
temp_mel.shape

(1, 49, 39, 1)

In [31]:
np.argmax(prediction)

0